In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_info import google_api_key

In [2]:
# Put constructed Data set into a csv for part 2 analysis
# Then drop unneeded column 
df = pd.read_csv('Extracted_Data.csv')
df = df.drop(['Unnamed: 0'],axis = 1)
df.head()

,City Name,Longitude,Latitude,Temperature,Humidity,Cloud Percent,Wind Speed (mph)
0,La Ronge,-105.2842,55.1001,52.05,62,20,3.44
1,Alta Floresta,-56.0861,-9.8756,78.44,37,27,1.92
2,Uyutnoye,33.3014,45.1994,63.82,77,40,9.24
3,Port Elizabeth,25.5701,-33.9180,56.53,87,2,8.99
4,Hobart,147.3294,-42.8794,52.45,58,40,3.00


In [3]:
# configure gmaps
gmaps.configure(api_key = google_api_key)

# set max inensity, data points, and weights to be placed on heat map
max_intensity = float(df['Humidity'].max())
locations = df[['Latitude','Longitude']]
weights = df['Humidity']

# Construct figure that layers will be added to and create the heatmap
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations,weights = weights,dissipating = False , max_intensity = max_intensity, point_radius = 1)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Construct the ideal conditions data frame
# Extract latitude and longitude pairs to be used for API requests
ideal_df = df[df['Temperature']<80][df['Temperature']>70][df['Wind Speed (mph)']<10][df['Cloud Percent'] == 0]
las = ideal_df['Latitude']
los = ideal_df['Longitude']
locs = list(zip(las,los))

/Users/borisrubel/opt/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [39]:
#base urls needed for api requests
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
base_url_id = "https://maps.googleapis.com/maps/api/place/details/json?"

# Parameters for URl construction
building = 'Hotel'
radius = '5000'

# Invalid index will be used to skip rows of cities not found in API calls when extracting the final data frame. 
# This is done to match all cities with extracted API data and keep them all at the size in order to construct the 
# hotel data frame 

hotel_data = []
invalid_index = []
x = 0

# Get all hotel information 
for coords in locs:
    url = base_url + f'location={coords[0]},{coords[1]}&radius={radius}&type={building}&key={google_api_key}'
    hotel_info = requests.get(url).json()

    try:
        hotel_data.append(hotel_info['results'][1])
    except:
        print(f'Could not find hotel near coordinates {coords[0]},{coords[1]} and index {x}!')
        invalid_index.append(x)
    x = x + 1


Could not find hotel near coordinates 30.3333,10.85 and index 7!


In [40]:
# update cities by skipping any invalid cities
valid_cities = [x for x in range(ideal_df.shape[0]) if x not in invalid_index]
h_cities = ideal_df['City Name'].iloc[valid_cities]

In [41]:
# First extract the country, longitude, latitude, and hotel name from Google Places API
h_lats = []
h_lons = []
h_names = []
h_countries = []

b_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
x = 0
for h in hotel_data:
    
    # use hotel's place ID to determine what country it is in
    # this is later matched by index to the city when constructing final hotel dataframe
    pid = h['place_id']
    url = b_url + f'place_id={pid}&key={google_api_key}'
    inf = requests.get(url).json()
    try:
        h_lats.append(h['geometry']['location']['lat'])
        h_lons.append(h['geometry']['location']['lng'])
        h_names.append(h['name'])
        
        # finds which entry of request contains the country name, then appends it to countries array
        c = [x['long_name'] for x in inf['result']['address_components'] if x['types'][0] == 'country']
        h_countries.append(c[0])
    
    except:
        print('ID not Valid')

# construct hotel dataframe
hotel_df = pd.DataFrame({'Hotel Name':h_names,'City':h_cities,
            'Country':h_countries,'Lat':h_lats,"Lng":h_lons})


In [36]:
# Display final hotel dataframe that will be used to create markers on fig (initialized google map figure above)
hotel_df

,Hotel Name,City,Country,Lat,Lng
26,Le gîte du Koniambo,Voh,New Caledonia,-20.983312,164.722214
156,Korla No.2 Elementary School,Korla,China,41.771057,86.161866
205,Jingu Hotel,Ürümqi,China,43.799718,87.609938
212,خانه شطرنج گنبدکاووس,Gonbad-e Kāvūs,Iran,37.241860,55.182100
256,Elysium Hotel,Chlórakas,Cyprus,34.772965,32.405619
279,بانک ملی ایران شعبه مرکزی ۱۴۴۸ داراب,Dārāb,Iran,28.752062,54.545818
292,Mantra Nelson Bay,Nelson Bay,Australia,-32.723499,152.145401
530,Hôtel Cap Macabou,Le Vauclin,Martinique,14.512985,-60.825912
553,Hotel Faro & Beach Club,Faro,Portugal,37.016432,-7.934978


In [37]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add markers to map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))